Imports

In [6]:
import pysolr
import json
import logging

Configuração do Logger

In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


Conexão com o Solr

In [8]:
logging.info("Conectando ao Solr...")
SOLR_URL_BASE = 'http://localhost:8983/solr/'
SOLR_CORE_NAME = 'documentos_ri' # Certifique-se que o nome do core está correto
SOLR_CONNECTION_STRING = f'{SOLR_URL_BASE}{SOLR_CORE_NAME}'

try:
    solr = pysolr.Solr(SOLR_CONNECTION_STRING, always_commit=True)
    solr.ping()
    logging.info(f"Conexão com o core '{SOLR_CORE_NAME}' bem-sucedida!")
except Exception as e:
    logging.error(f"Não foi possível conectar ao Solr. Verifique se o serviço está ativo. Detalhe: {e}")
    exit()

2025-07-22 09:39:49,654 - INFO - Conectando ao Solr...
2025-07-22 09:39:49,664 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/admin/ping/?' (get) with body '' in 0.007 seconds, with status 200
2025-07-22 09:39:49,664 - INFO - Conexão com o core 'documentos_ri' bem-sucedida!


Carregamento dos Documentos

In [9]:
logging.info("Carregando o arquivo 'map_documentos.json'...")
try:
    with open('map_documentos.json', 'r', encoding='utf-8') as f:
        map_docs = json.load(f)
    logging.info(f"Foram carregados {len(map_docs)} documentos.")
except FileNotFoundError:
    logging.error("Arquivo 'map_documentos.json' não encontrado. Abortando.")
    exit()

2025-07-22 09:39:49,682 - INFO - Carregando o arquivo 'map_documentos.json'...
2025-07-22 09:39:49,818 - INFO - Foram carregados 162259 documentos.


Processo de Indexação

In [10]:
logging.info("Iniciando o processo de indexação...")

# 1. Limpa todos os documentos existentes no core para evitar duplicatas
logging.info("Limpando documentos antigos do core...")
solr.delete(q='*:*')
logging.info("Core limpo.")

# 2. Formata os documentos para o formato que o Solr espera
# O campo 'id' é o ID do documento e 'texto_documento' é o conteúdo.
# IMPORTANTE: O schema do Solr deve ter um campo chamado 'texto_documento'.
documentos_para_indexar = []
for doc_id, texto in map_docs.items():
    documentos_para_indexar.append({
        "id": doc_id,
        "texto_documento": texto
    })

# 3. Envia os documentos para o Solr indexar em lotes (mais eficiente)
logging.info(f"Enviando {len(documentos_para_indexar)} documentos para indexação...")
try:
    solr.add(documentos_para_indexar, commit=True)
    logging.info("Indexação concluída com sucesso!")
except Exception as e:
    logging.error(f"ERRO durante a indexação: {e}")
    logging.error("Verifique o schema do Solr e se os campos (ex: 'id', 'texto_documento') existem.")

2025-07-22 09:39:49,831 - INFO - Iniciando o processo de indexação...
2025-07-22 09:39:49,832 - INFO - Limpando documentos antigos do core...
2025-07-22 09:39:49,876 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/update/?commit=true' (post) with body '<delete><q' in 0.043 seconds, with status 200
2025-07-22 09:39:49,877 - INFO - Core limpo.
2025-07-22 09:39:49,937 - INFO - Enviando 162259 documentos para indexação...
2025-07-22 09:39:53,971 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/update/?commit=true' (post) with body '[{"id": "0' in 3.741 seconds, with status 200
2025-07-22 09:39:53,975 - INFO - Indexação concluída com sucesso!
